In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd
import folium
from folium.plugins import HeatMap

In [2]:
def get_lat_lon_from_address(address_l):
    url = 'http://www.geocoding.jp/api/'
    latlons = []
    for address in tqdm(address_l):
        payload = {"v": 1.1, 'q': address}
        r = requests.get(url, params=payload)
        ret = BeautifulSoup(r.content,'lxml')
        if ret.find('error'):
            continue
        else:
            lat = ret.find('lat').string
            lon = ret.find('lng').string
            latlons.append([lat,lon])
            time.sleep(10)
    return latlons

In [2]:
tokyo = pd.read_csv("datas/tokyo_mu.csv")
kanagawa = pd.read_csv("datas/kanagawa_mu.csv")
saitama = pd.read_csv("datas/saitama_mu.csv")

In [3]:
#読み込んだデータから住所を抽出してリスト化
kanagawa_add_li = kanagawa['9'].str.cat(kanagawa['10'].str.cat(kanagawa['11'])).to_list()
saitama_add_li = saitama['9'].str.cat(saitama['10'].str.cat(saitama['11'])).to_list()
tokyo_add_li = tokyo['9'].str.cat(tokyo['10'].str.cat(tokyo['11'])).to_list()

In [ ]:
#http://www.geocoding.jp/api/を利用利して住所データから緯度経度を取得
kanagawa_geo = get_lat_lon_from_address(kanagawa_add_li)
saitama_geo = get_lat_lon_from_address(saitama_add_li)
tokyo_geo = get_lat_lon_from_address(tokyo_add_li)

In [12]:
#3都県の緯度経度を結合
all_geo = kanagawa_geo + saitama_geo + tokyo_geo

In [62]:
#all_geoの座標データの密集度合いをheatmapで可視化
map_heat = folium.Map(location=[35.7836568342218, 139.61961339462664], zoom_start=11)
HeatMap(all_geo, radius=50, blur=50, max_opacity=0.01).add_to(map_heat)

In [64]:
#マップを表示
map_heat

In [65]:
#マップを保存
map_heat.save("heat_map.html")